# REGRESIÓN LINEAL MÚLTIPLE


Crea el fichero u02_p03_a1_reglin_multi_<tus_iniciales>.py. Utiliza pandas y carga los datos del
fichero "50_startups.csv". Utilizaremos como predictoras las columnas "R&D Spend", "Marqueting
Spend" y "State" y como target usaremos "Profit" (beneficios). Imprime los 5 primeros ejemplos de las
predictoras: <br><br>
Para realizar regresión lineal no podemos usar una columna categórica como "State". Codifica sus
valores usando el método one-hot-encoder de manera que aparecerán 3 nuevas columnas que
corresponden a cada uno de los valores que puede tener la columna original (‘New York’, ‘California’,
‘Florida’) y sus valores estarán a 1 cuando sea ese valor y a 0 cuando no. En vez de hacerlo a mano,
vamos a utilizar los objetos sklearn.compose.ColumnTransformer y
sklearn.preprocessing.OneHotEncoder. Busca información de como hacerlo. Tras realizarlo imprime
las primeras 5 filas de los datos transformados.<br><br>
Como todo es un proceso aleatorio, para poder obtener resultados comparables utilizamos la semilla
"123" en todos los procesos. Divide los datos en train y test dejando el 80% para entrenamiento.
Imprime los 5 primeros ejemplos de X_train, y_train, X_test e y_test:

## 1. Imporación de librerías

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [79]:
df = pd.read_csv("recursos/U02_P03/50_Startups.csv")

In [80]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [81]:
np.random.seed(123)

In [82]:
# Definir las columnas predictoras y la variable objetivo
columnas_numericas = ["R&D Spend", "Marketing Spend"]
columnas_categoricas = ["State"]
X = df[columnas_numericas + columnas_categoricas]
y = df["Profit"]

In [83]:
# Imprimir los 5 primeros ejemplos de las predictoras
print("Primeras 5 filas de X antes de transformación:")
print(X.head())

Primeras 5 filas de X antes de transformación:
   R&D Spend  Marketing Spend       State
0  165349.20        471784.10    New York
1  162597.70        443898.53  California
2  153441.51        407934.54     Florida
3  144372.41        383199.62    New York
4  142107.34        366168.42     Florida


In [84]:
# Aplicar OneHotEncoder a la columna categórica
preprocessor = ColumnTransformer(
    transformers=[("onehot", OneHotEncoder(drop='first'), columnas_categoricas)],
    remainder="passthrough"
)

# Transformar los datos
X_transformed = preprocessor.fit_transform(X)

# Convertir a DataFrame con nombres de columnas
nombres_columnas = preprocessor.get_feature_names_out()
X_transformed = pd.DataFrame(X_transformed, columns=nombres_columnas)

# Imprimir las primeras 5 filas de los datos transformados
print("\nPrimeras 5 filas de X después de transformación:")
print(X_transformed.head())


Primeras 5 filas de X después de transformación:
   onehot__State_Florida  onehot__State_New York  remainder__R&D Spend  \
0                    0.0                     1.0             165349.20   
1                    0.0                     0.0             162597.70   
2                    1.0                     0.0             153441.51   
3                    0.0                     1.0             144372.41   
4                    1.0                     0.0             142107.34   

   remainder__Marketing Spend  
0                   471784.10  
1                   443898.53  
2                   407934.54  
3                   383199.62  
4                   366168.42  


In [85]:
# Dividir los datos en train (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=SEED)

# Imprimir los primeros 5 ejemplos de cada conjunto
print("\nPrimeras 5 filas de X_train:")
print(X_train.head())
print("\nPrimeras 5 filas de y_train:")
print(y_train.head())
print("\nPrimeras 5 filas de X_test:")
print(X_test.head())
print("\nPrimeras 5 filas de y_test:")
print(y_test.head())


Primeras 5 filas de X_train:
    onehot__State_Florida  onehot__State_New York  remainder__R&D Spend  \
21                    0.0                     1.0              78389.47   
47                    0.0                     0.0                  0.00   
11                    0.0                     0.0             100671.96   
41                    1.0                     0.0              27892.92   
5                     0.0                     1.0             131876.90   

    remainder__Marketing Spend  
21                   299737.29  
47                        0.00  
11                   249744.55  
41                   164470.71  
5                    362861.36  

Primeras 5 filas de y_train:
21    111313.02
47     42559.73
11    144259.40
41     77798.83
5     156991.12
Name: Profit, dtype: float64

Primeras 5 filas de X_test:
    onehot__State_Florida  onehot__State_New York  remainder__R&D Spend  \
10                    1.0                     0.0             101913.08   
13 

In [86]:
# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Calcular el score para datos de entrenamiento y prueba
train_score = modelo.score(X_train, y_train)
test_score = modelo.score(X_test, y_test)

print("\nScore en datos de entrenamiento:", train_score)
print("Score en datos de prueba:", test_score)


Score en datos de entrenamiento: 0.944872235399549
Score en datos de prueba: 0.965754198932209


### ENTREGA 1: Muestra:
a) Capturas de ejecución y el código.<br><br>
b) ¿Qué significa en este caso que el score sea aproximadamente del 0.966 en los datos de test?
<p style="color:green;">Obteniendo un score de 0.96 podemos concluir que el modelo es bastante aceptable para la predicción</p>


In [87]:
# Funciones para calcular SSE, SST y SSR
def calculate_SSE(y_true, y_pred):
    """ Calcula el SSE (Sum of Squared Errors) """
    return np.sum((y_true - y_pred) ** 2)

def calculate_SST(y_true):
    """ Calcula el SST (Total Sum of Squares) """
    return np.sum((y_true - np.mean(y_true)) ** 2)

def calculate_SSR(SST, SSE):
    """ Calcula el SSR (Sum of Squares for Regression) """
    return SST - SSE

# Calcular las métricas para los datos de entrenamiento
y_train_pred = modelo.predict(X_train)
SSE_train = calculate_SSE(y_train, y_train_pred)
SST_train = calculate_SST(y_train)
SSR_train = calculate_SSR(SST_train, SSE_train)

# Calcular R² para entrenamiento
R2_train = 1 - SSE_train / SST_train

# Calcular R² ajustado para entrenamiento
n_train = len(y_train)
k_train = X_train.shape[1]
R2_adj_train = 1 - (SSE_train / (n_train - k_train - 1)) / (SST_train / (n_train - 1))

In [88]:
# Imprimir las métricas para los datos de entrenamiento
print(f"Entrenamiento:")
print(f"SSE: {SSE_train}")
print(f"SST: {SST_train}")
print(f"SSR: {SSR_train}")
print(f"R²: {R2_train}")
print(f"R² ajustado: {R2_adj_train}")

Entrenamiento:
SSE: 3489621036.392025
SST: 63300608353.770935
SSR: 59810987317.37891
R²: 0.944872235399549
R² ajustado: 0.9385719194452118


In [89]:
# Calcular las métricas para los datos de prueba
y_test_pred = modelo.predict(X_test)
SSE_test = calculate_SSE(y_test, y_test_pred)
SST_test = calculate_SST(y_test)
SSR_test = calculate_SSR(SST_test, SSE_test)

# Calcular R² para prueba
R2_test = 1 - SSE_test / SST_test

# Calcular R² ajustado para prueba
n_test = len(y_test)
k_test = X_test.shape[1]
R2_adj_test = 1 - (SSE_test / (n_test - k_test - 1)) / (SST_test / (n_test - 1))

In [90]:
# Imprimir las métricas para los datos de prueba
print(f"\nPrueba:")
print(f"SSE: {SSE_test}")
print(f"SST: {SST_test}")
print(f"SSR: {SSR_test}")
print(f"R²: {R2_test}")
print(f"R² ajustado: {R2_adj_test}")


Prueba:
SSE: 516139212.6673195
SST: 15071605761.115088
SSR: 14555466548.447767
R²: 0.965754198932209
R² ajustado: 0.9383575580779763


In [91]:
# Incluir "Administration" como una nueva característica
columnas_nuevas = ["R&D Spend", "Administration", "Marketing Spend", "State"]

# Crear el nuevo DataFrame con las características
X_nuevas = df[columnas_nuevas]
y_nuevos = df['Profit']

# Dividir nuevamente los datos en entrenamiento y prueba
X_train_nuevas, X_test_nuevas, y_train_nuevos, y_test_nuevos = train_test_split(X_nuevas, y_nuevos, test_size=0.2, random_state=123)

# Crear un nuevo pipeline para el modelo con las nuevas características
preprocessor_nuevas = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["State"]),
    ],
    remainder="passthrough"
)

# Crear el pipeline
pipeline_nuevas = Pipeline(steps=[
    ('preprocessor', preprocessor_nuevas),
    ('regressor', LinearRegression())
])

# Entrenar el modelo con las nuevas características
pipeline_nuevas.fit(X_train_nuevas, y_train_nuevos)

C:\Users\ESP\Desktop\MachineLearning\saa\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['State'])])),
                ('regressor', LinearRegression())])

In [92]:
# Calcular y mostrar las métricas para el nuevo modelo
y_train_pred_nuevas = pipeline_nuevas.predict(X_train_nuevas)
SSE_train_nuevas = calculate_SSE(y_train_nuevos, y_train_pred_nuevas)
SST_train_nuevas = calculate_SST(y_train_nuevos)
SSR_train_nuevas = calculate_SSR(SST_train_nuevas, SSE_train_nuevas)
R2_train_nuevas = 1 - SSE_train_nuevas / SST_train_nuevas
R2_adj_train_nuevas = 1 - (SSE_train_nuevas / (len(y_train_nuevos) - X_train_nuevas.shape[1] - 1)) / (SST_train_nuevas / (len(y_train_nuevos) - 1))

In [93]:
print(f"\nNuevo modelo de entrenamiento:")
print(f"SSE: {SSE_train_nuevas}")
print(f"SST: {SST_train_nuevas}")
print(f"SSR: {SSR_train_nuevas}")
print(f"R²: {R2_train_nuevas}")
print(f"R² ajustado: {R2_adj_train_nuevas}")


Nuevo modelo de entrenamiento:
SSE: 3481749147.2060595
SST: 63300608353.770935
SSR: 59818859206.56487
R²: 0.9449965926433526
R² ajustado: 0.9387104889454501


In [94]:
# Hacer lo mismo para los datos de prueba
y_test_pred_nuevas = pipeline_nuevas.predict(X_test_nuevas)
SSE_test_nuevas = calculate_SSE(y_test_nuevos, y_test_pred_nuevas)
SST_test_nuevas = calculate_SST(y_test_nuevos)
SSR_test_nuevas = calculate_SSR(SST_test_nuevas, SSE_test_nuevas)
R2_test_nuevas = 1 - SSE_test_nuevas / SST_test_nuevas
R2_adj_test_nuevas = 1 - (SSE_test_nuevas / (len(y_test_nuevos) - X_test_nuevas.shape[1] - 1)) / (SST_test_nuevas / (len(y_test_nuevos) - 1))

In [95]:
print(f"\nNuevo modelo de prueba:")
print(f"SSE: {SSE_test_nuevas}")
print(f"SST: {SST_test_nuevas}")
print(f"SSR: {SSR_test_nuevas}")
print(f"R²: {R2_test_nuevas}")
print(f"R² ajustado: {R2_adj_test_nuevas}")


Nuevo modelo de prueba:
SSE: 500379591.6416779
SST: 15071605761.115088
SSR: 14571226169.47341
R²: 0.9667998486974319
R² ajustado: 0.9402397276553774


### ENTREGA 2: Muestra:

a) Capturas de ejecución y el código. <br><br>
b) Haz una marca donde aumente el R2
: [ ]Train [X]Test <br><br>
c) Haz una marca donde aumente del R2
 ajustado: [ ]Train [X]Test <br><br>
d) ¿Dónde aparece mayor mejora? En el R2
 o en el R2
 ajustado. <p style="color: green;">En el r2</p><br><br>
e) ¿Marca lo que calcula score() en scikit-learn? (_)SSE (_)SST (_)SSR (X)R
2 (_)R
2 ajustado

K-NN <br><br>
Como tenemos pocos datos vamos a ver como se comportará un K-NN, por ejemplo un 3-NN. A
continuación del fichero, añade las sentencias para crear uno y muestra su score para los datos de test
tras entrenarlo. Usaremos todas las columnas de predictoras:

In [97]:
from sklearn.neighbors import KNeighborsRegressor
# Crear y entrenar el modelo K-NN (3 vecinos)
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

# Calcular el score del modelo K-NN para datos de prueba
knn_test_score = knn.score(X_test, y_test)
print("\nScore de K-NN en datos de prueba:", knn_test_score)


Score de K-NN en datos de prueba: 0.7162220513708704


### ENTREGA 3: Muestra:
a) Capturas de ejecución y el código.<br><br>
b) Valor del hiperparámetro K que mejore el score alcanzado con K=3.
<p style="color:green">0.71</p>
